# Gemma-2b 활용/ 요약 모델 만들기

In [7]:
# import os
# # 서버의 GPU 1번 사용하게 만들기 
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # 1번 GPU만 노출됨
import torch

# 원하는 GPU를 지정 (예: 두 번째 GPU 사용)
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda:1


In [1]:
import pandas as pd

# ✅ CSV 파일 불러오기
csv_path = "extracted_documents.csv"
df = pd.read_csv(csv_path, encoding="utf-8")

# ✅ 실제 컬럼명에 맞게 변경: "original_text" → "input_text", "summary_text" → "target_text"
df = df.rename(columns={"original_text": "input_text", "summary_text": "target_text"})

# ✅ 변경된 컬럼명 확인
print("변경된 데이터셋 컬럼명:", df.columns)
print("데이터 샘플:")
print(df.head())

# ✅ 변경된 데이터 저장 (원본 유지하려면 다른 이름으로 저장)
df.to_csv("extracted_documents_updated.csv", index=False, encoding="utf-8-sig")  # ✅ utf-8-sig: 한글 깨짐 방지


변경된 데이터셋 컬럼명: Index(['input_text', 'target_text'], dtype='object')
데이터 샘플:
                                          input_text  \
0  원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해...   
1  수출입업체인 원고가 의류제품을 제조ㆍ수출함에 있어 같은 그룹내 종합무역상사인 소외 ...   
2  가등기담보권자가 제소전 화해조항에 따라 자기 명의로 소유권이전의 본등기를 경료한 후...   
3  가. 부가가치세법 제22조 제3항 단서에 제1호와 제2호가 동시에 해당한다는 뜻은 ...   
4  소득세법 제116조 제1항의 규정에 의하면 정부는 과세표준확정신고를 하여야 할 자에...   

                                         target_text  
0  원고가  주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에 ...  
1  수출입업체인 원고가 의류제품을 제조ㆍ수출함에 있어 소외 회사의 직수출실적을 지원하기...  
2  가등기담보권자가 제소전 화해조항에 의해 자기 명의로 소유권이전의 본등기를 경료하고 ...  
3  부가가치세법 제22조 제3항 단서에 제1호와 제2호가 동시에 해당한다는 의미는 제1...  
4  소득세법 제116조 제1항에 따르면 정부는 과세표준확정신고를 해야 할 자에 대해 당...  


## 📌 1. 모델 및 토크나이저 로드 (QLoRA 적용)

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from main import access_token  # ✅ main.py에서 Hugging Face API 토큰 가져오기

def load_model_and_tokenizer(model_name="google/gemma-2-2b-it", quantized=True):
    """
    Hugging Face에서 Gemma 모델과 토크나이저를 불러옵니다.
    - `main.py`에서 저장한 Hugging Face API 토큰을 사용하여 인증합니다.
    - `quantized=True`일 경우, 4-bit QLoRA 양자화를 적용하여 메모리 효율을 극대화합니다.
    """
    token = get_huggingface_token()  # ✅ API 토큰 불러오기

    # ✅ 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(model_name, token=token)

    if quantized:
        # ✅ 4-bit QLoRA 설정
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,  # 16-bit 연산 사용
            bnb_4bit_quant_type="nf4",  # QLoRA 최적화
            bnb_4bit_use_double_quant=True  # 이중 양자화 적용
        )
        device_map = {"": "cuda:1"} if torch.cuda.is_available() else "cpu"  # ✅ GPU 설정

        # ✅ 모델 로드 (QLoRA 적용)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map=device_map,
            token=token  # ✅ API 인증 적용
        )
    else:
        # ✅ 일반 모델 로드 (양자화 X)
        model = AutoModelForCausalLM.from_pretrained(model_name, token=token)

    return model, tokenizer

# ✅ 사용 예시:
# model, tokenizer = load_model_and_tokenizer("google/gemma-2-2b-it", quantized=True)


## 📌 2. 데이터셋 불러오기 및 포맷팅

In [3]:
import pandas as pd
from datasets import Dataset

def prepare_dataset(csv_path, encoding="utf-8"):
    """
    CSV 파일을 불러와서 Hugging Face Dataset 형식으로 변환합니다.
    'original_text'와 'summary_text' 컬럼을 'input_text'와 'target_text'로 변환합니다.
    """
    df = pd.read_csv(csv_path, encoding=encoding)

    # ✅ 컬럼명 자동 확인 및 변경
    if "original_text" in df.columns or "summary_text" in df.columns:
        df = df.rename(columns={"original_text": "input_text", "summary_text": "target_text"})
    elif "input_text" not in df.columns or "target_text" not in df.columns:
        raise ValueError(f"CSV 파일에 필요한 컬럼이 없습니다. 현재 컬럼: {df.columns}")

    dataset = Dataset.from_pandas(df)
    return dataset


## 📌 3. 데이터 토큰화

In [4]:
def tokenize_dataset(dataset, tokenizer, max_input_length=1024, max_target_length=256):
    """
    데이터셋을 토큰화하여 모델이 학습할 수 있는 형태로 변환합니다.
    """
    def tokenize_function(example):
        model_inputs = tokenizer(
            example["input_text"],
            max_length=max_input_length,
            truncation=True,
            padding="max_length"
        )
        labels = tokenizer(
            text_target=example["target_text"],  # ✅ text_target으로 변경
            max_length=max_target_length,
            truncation=True,
            padding="max_length"
        )
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)
    return tokenized_dataset


## 📌 4. Fine-Tuning (QLoRA 적용)

### QLoRa 적용

In [5]:
from peft import LoraConfig, get_peft_model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

def load_model_and_tokenizer(model_name="google/gemma-2-2b-it"):
    """
    QLoRA 기반 모델 및 토크나이저를 로드하고, LoRA 어댑터를 적용하는 함수.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # ✅ 4-bit 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,  # FP16 연산 사용
        bnb_4bit_quant_type="nf4",  # QLoRA 최적화된 nf4 양자화 적용
        bnb_4bit_use_double_quant=True  # 이중 양자화 활성화
    )

    # ✅ GPU 설정
    device_map = {"": "cuda:1"} if torch.cuda.is_available() else "cpu"

    # ✅ 양자화된 모델 로드
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map=device_map
    )

    # ✅ QLoRA 어댑터 설정
    lora_config = LoraConfig(
        r=16,  # 랭크 값 (적절한 차원 크기, 클수록 정확도 증가)
        lora_alpha=32,  # LoRA 스케일링 파라미터
        target_modules=["q_proj", "v_proj"],  # QLoRA 적용 대상 모듈
        lora_dropout=0.05,  # 드롭아웃 비율
        bias="none",
        task_type="CAUSAL_LM"  # 인과적 언어 모델 (Causal LM)
    )

    # ✅ LoRA 적용된 PEFT 모델 생성
    model = get_peft_model(model, lora_config)

    return model, tokenizer


### Fine-Tuning

In [6]:
import torch
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

def fine_tune_model(model, tokenizer, tokenized_dataset, output_dir="./results", epochs=3, batch_size=1):
    """
    QLoRA 기반 Fine-Tuning 실행 함수.
    """
    # ✅ 데이터셋 분할: 학습(train) & 검증(eval) 데이터셋으로 분리
    split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
    train_dataset = split_dataset["train"]
    eval_dataset = split_dataset["test"]
    
    # ✅ 데이터 콜레이터: 패딩 자동 처리
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    # ✅ 학습 파라미터 설정 (QLoRA 적용)
    training_args = TrainingArguments(
        output_dir="./fine_tuned_results",
        evaluation_strategy="epoch",
        learning_rate=3e-4,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=4,  # ✅ Gradient Accumulation 활성화
        num_train_epochs=3,
        weight_decay=0.01,
        fp16=True,
        save_total_limit=1,
        logging_steps=50,
        seed=42,
        deepspeed="deepspeed_config.json"  # ✅ DeepSpeed 적용
)


    # ✅ Trainer 객체 생성
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator
    )

    # ✅ Fine-Tuning 실행
    trainer.train()

    # ✅ 학습 완료된 모델 저장
    trainer.save_model(output_dir)


## 5. 메인 함수: 전체 파이프라인 실행

In [8]:
import os
import torch

def main():
    # ✅ 환경 변수 설정 (메모리 단편화 완화)
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
    
    # ✅ GPU 설정
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)
    
    # ✅ 모델 및 토크나이저 로드 (QLoRA 적용)
    model, tokenizer = load_model_and_tokenizer("google/gemma-2-2b-it")
    
    # ✅ 데이터 로드 및 토큰화
    csv_path = "extracted_documents.csv"
    raw_dataset = prepare_dataset(csv_path, encoding="utf-8")
    tokenized_dataset = tokenize_dataset(raw_dataset, tokenizer)
    
    # ✅ Fine-Tuning 실행
    fine_tune_model(model, tokenizer, tokenized_dataset, output_dir="./fine_tuned_results")

if __name__ == "__main__":
    main()


Using device: cuda:1


OSError: There was a specific connection error when trying to load google/gemma-2-2b-it:
401 Client Error: Unauthorized for url: https://huggingface.co/google/gemma-2-2b-it/resolve/main/config.json (Request ID: Root=1-67c41a6f-4a4240ad05f575f74e171a6b;6786b343-f2fb-46e7-9cca-ef38c0e5efce)

Invalid credentials in Authorization header

## 📌 6. 성능 평가 (요약 생성)

In [1]:
def generate_summary(input_text, model, tokenizer, max_new_tokens=150):
    """
    Fine-Tuned 모델을 이용해 요약을 생성하는 함수.
    """
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.8,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# ✅ 사용 예시:
sample_input = "한국 경제는 글로벌 경기 둔화와 내수 부진 등으로 어려움을 겪고 있으며, 정부는 이를 극복하기 위해 다양한 정책을 발표하고 있다."
sample_summary = generate_summary(sample_input, model, tokenizer)
print("\n✅ 생성된 요약:")
print(sample_summary)


NameError: name 'model' is not defined

## 6. 출력 결과 확인 후 프롬프트 및 하이퍼 파라미터 조정하기

In [ ]:
# def evaluate_and_adjust():
#     """
#     생성된 결과를 평가하고, 프롬프트 문구나 생성 파라미터를 조정하는 반복 과정을 수행합니다.
#     이 단계는 수동 또는 자동화된 평가 루프를 통해 진행할 수 있습니다.
#     """
#     # 예시: 생성 결과 평가 (실제 평가 기준은 ROUGE, BLEU 등 메트릭을 활용할 수 있음)
#     print("출력 결과를 확인하고, 프롬프트 및 파라미터를 조정하세요.")
#     # 필요 시, 재생성 및 파라미터 변경 로직을 추가합니다.

# # 평가 및 조정 호출 예시
# evaluate_and_adjust()


## 7. 튜닝 루프 만들기

In [ ]:
# def tuning_loop(model, tokenizer, trainer, iterations=3):
#     """
#     4~6단계를 반복하는 튜닝 루프입니다.
#     iterations 값은 실험에 따라 조정합니다.
#     각 반복에서 모델의 출력 결과를 평가하고,
#     필요 시 프롬프트 문구 및 하이퍼파라미터를 조정하여 최적의 결과를 도출합니다.
#     """
#     for i in range(iterations):
#         print(f"\n=== Tuning iteration {i+1} ===")
#         # 프롬프트 및 하이퍼파라미터 설정
#         sample_input = (
#             "### 원문:\n"
#             "최근 한국 경제는 글로벌 경기 둔화와 내수 부진 등으로 어려움을 겪고 있으며, "
#             "정부와 기업들은 다양한 정책과 혁신을 통해 위기를 극복하고자 노력하고 있습니다.\n\n"
#             "### 요약:"
#         )
#         output = generate_summary(sample_input, model, tokenizer, max_new_tokens=150)
#         print("Generated Output:\n", output)
        
#         # 평가 및 필요 시 파라미터, 프롬프트 조정 (여기서는 단순히 출력 확인)
#         evaluate_and_adjust()
    
#     print("최종 튜닝 완료.")

# # 튜닝 루프 실행 예시
# tuning_loop(model, tokenizer, trainer, iterations=3)


: 